
## Task 1: Binary Representations

In [ ]:
def rotl(x, n=1):
    """
    Rotates the bits in a 32-bit unsigned integer to the left by n places.
    """
    n = n % 32  # Ensure n is within valid bit range (0-31)
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

In [ ]:
def rotr(x, n=1):
    """
    Rotates the bits in a 32-bit unsigned integer to the right by n places.
    """
    n = n % 32  # Ensure n is within valid bit range (0-31)
    return (x >> n) | ((x << (32 - n)) & 0xFFFFFFFF)

In [ ]:
def ch(x, y, z):
    """
    Chooses bits from y where x has bits set to 1, and from z where x has bits set to 0.
    Returns:
    int: Resulting 32-bit integer after bitwise choice
    """
    return (x & y) | (~x & z)  # If x bit is 1 -> choose from y, else from z


## Task 2: Hash Functions

## Task 3: SHA256

## Task 4: Prime Numbers

## Task 5: Roots

## Task 6: Proof of Work

## Task 7: Turing Machines

## Task 8: Computational Complexity